In [9]:
import pandas as pd
# 读取 CSV，此时保留原始 header（第 0 行）
config = pd.read_csv("/Users/shiyihe/Desktop/VIS_RESULT/m_g_config.csv", header=0)
# 先把所有列名左右的空格或不可见字符去掉
config.columns = config.columns.str.strip()
config = config.rename(columns={
    "Participant ID":          "participantId",
    "Task":                    "task",
    "Semantic Error":          "wrong_field_value",
    "Unnamed: 3":              "incomplete_modification",
    "Unnamed: 4":              "inappropriate_modification",
    "Unnamed: 5":              "redundant_field_value",
    "Syntax Error":            "fully_correct",
    "Unnamed: 7":              "repairable_error",
    "Unnamed: 8":              "critical_error"
})


config = config.drop(index=0).reset_index(drop=True)

def add_error_scores(df: pd.DataFrame) -> pd.DataFrame:
    """
    输入 df 需要包含列：
      - 'wrong_field_value'
      - 'incomplete_modification'
      - 'inappropriate_modification'
      - 'redundant_field_value'
      - 'Syntax Error'
      - 'repairable_error'
    输出会多出三列：
      - semantic_score
      - syntax_score
      - score = 0.5*(semantic_score + syntax_score)
    """
    df = df.copy()
    
    # 帮助函数，把各种 TRUE / FALSE / N 标准化为布尔
    def to_bool(x):
        if pd.isna(x): return False
        if isinstance(x, bool): return x
        s = str(x).strip().upper()
        return s in ('TRUE', 'T', '1')
    
    # 1. semantic_score
    # 如果 'Semantic Error'=='N' → semantic_score=0
    # 否则 count = 4 个语义相关列中 True 的个数
    # semantic_score = (4 - count) / 4
    sem_cols = [
        'wrong_field_value',
        'incomplete_modification',
        'inappropriate_modification',
        'redundant_field_value'
    ]
    # 先把所有相关列转为布尔矩阵
    bool_sem = df[sem_cols].applymap(to_bool)
    # 记录哪些行 semantic_error 为 'N'
    mask_no_sem = df['wrong_field_value'].astype(str).str.upper() == 'N'
    # 计算 count
    count_true = bool_sem.sum(axis=1)
    # 计算 semantic_score
    df['semantic_score'] = np.where(
        mask_no_sem,
        0.0,
        (4 - count_true) / 4
    )
    
    # 2. syntax_score
    # 如果 semantic_error=='N' → syntax_score=0
    # 否则：
    #   如果 Syntax Error 为 True → 1
    #   elif repairable_error 为 True → 0.5
    #   else → 0
    mask_syntax = df['fully_correct'].apply(to_bool)
    mask_repair = df['repairable_error'].apply(to_bool)
    df['syntax_score'] = np.where(
        mask_no_sem,
        0.0,
        np.where(
            mask_syntax, 
            1.0,
            np.where(mask_repair, 0.5, 0.0)
        )
    )
    
    # 3. 最终 score
    df['score'] = 0.5 * (df['semantic_score'] + df['syntax_score'])
    
    return df

# 用法示例
# new_df = add_error_scores(old_df)
config = add_error_scores(config)
config = config.copy()

# 按 task 分组，计算组内 score 的 z-score
config['score_z'] = config.groupby('task')['score'] \
                  .transform(lambda x: (x - x.mean()) / x.std(ddof=0))
config=config[['participantId','task','semantic_score','syntax_score','score','score_z']]
config.to_csv('m_config_score.csv',index=False)
config



,participantId,task,semantic_score,syntax_score,score,score_z
0,67895b7d6400163124042af7,m_c_1,0.00,0.0,0.000,-3.491460
1,67895b7d6400163124042af7,m_c_2,0.00,0.0,0.000,-2.635476
2,67895b7d6400163124042af7,m_c_3,0.00,0.0,0.000,-2.844370
3,67895b7d6400163124042af7,m_c_4,0.00,0.0,0.000,-2.046031
4,659960f32faa4ff4b116de92,m_c_1,1.00,1.0,1.000,0.574785
...,...,...,...,...,...,...
423,67d24179c5bd87691f1fe039,m_c_4,0.75,1.0,0.875,0.884136
424,5c4f5967aac8be0001716a65,m_c_1,1.00,1.0,1.000,0.574785
425,5c4f5967aac8be0001716a66,m_c_2,1.00,1.0,1.000,0.829912
426,5c4f5967aac8be0001716a67,m_c_3,1.00,1.0,1.000,0.889956


In [7]:
import pandas as pd
import numpy as np

def add_error_scores(df: pd.DataFrame) -> pd.DataFrame:
    """
    输入 df 需要包含列：
      - 'wrong_field_value'
      - 'incomplete_modification'
      - 'inappropriate_modification'
      - 'redundant_field_value'
      - 'Syntax Error'
      - 'repairable_error'
    输出会多出三列：
      - semantic_score
      - syntax_score
      - score = 0.5*(semantic_score + syntax_score)
    """
    df = df.copy()
    
    # 帮助函数，把各种 TRUE / FALSE / N 标准化为布尔
    def to_bool(x):
        if pd.isna(x): return False
        if isinstance(x, bool): return x
        s = str(x).strip().upper()
        return s in ('TRUE', 'T', '1')
    
    # 1. semantic_score
    # 如果 'Semantic Error'=='N' → semantic_score=0
    # 否则 count = 4 个语义相关列中 True 的个数
    # semantic_score = (4 - count) / 4
    sem_cols = [
        'wrong_field_value',
        'incomplete_modification',
        'inappropriate_modification',
        'redundant_field_value'
    ]
    # 先把所有相关列转为布尔矩阵
    bool_sem = df[sem_cols].applymap(to_bool)
    # 记录哪些行 semantic_error 为 'N'
    mask_no_sem = df['wrong_field_value'].astype(str).str.upper() == 'N'
    # 计算 count
    count_true = bool_sem.sum(axis=1)
    # 计算 semantic_score
    df['semantic_score'] = np.where(
        mask_no_sem,
        0.0,
        (4 - count_true) / 4
    )
    
    # 2. syntax_score
    # 如果 semantic_error=='N' → syntax_score=0
    # 否则：
    #   如果 Syntax Error 为 True → 1
    #   elif repairable_error 为 True → 0.5
    #   else → 0
    mask_syntax = df['fully_correct'].apply(to_bool)
    mask_repair = df['repairable_error'].apply(to_bool)
    df['syntax_score'] = np.where(
        mask_no_sem,
        0.0,
        np.where(
            mask_syntax, 
            1.0,
            np.where(mask_repair, 0.5, 0.0)
        )
    )
    
    # 3. 最终 score
    df['score'] = 0.5 * (df['semantic_score'] + df['syntax_score'])
    
    return df

# 用法示例
# new_df = add_error_scores(old_df)
config = add_error_scores(config)
config = config.copy()

# 按 task 分组，计算组内 score 的 z-score
config['score_z'] = config.groupby('task')['score'] \
                  .transform(lambda x: (x - x.mean()) / x.std(ddof=0))

config


,participantId,task,wrong_field_value,incomplete_modification,inappropriate_modification,redundant_field_value,fully_correct,repairable_error,critical_error,semantic_score,syntax_score,score,score_z
0,67895b7d6400163124042af7,m_c_1,N,N,N,N,N,N,N,0.00,0.0,0.000,-3.491460
1,67895b7d6400163124042af7,m_c_2,N,N,N,N,N,N,N,0.00,0.0,0.000,-2.635476
2,67895b7d6400163124042af7,m_c_3,N,N,N,N,N,N,N,0.00,0.0,0.000,-2.844370
3,67895b7d6400163124042af7,m_c_4,N,N,N,N,N,N,N,0.00,0.0,0.000,-2.046031
4,659960f32faa4ff4b116de92,m_c_1,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,1.00,1.0,1.000,0.574785
...,...,...,...,...,...,...,...,...,...,...,...,...,...
423,67d24179c5bd87691f1fe039,m_c_4,TRUE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,0.75,1.0,0.875,0.884136
424,5c4f5967aac8be0001716a65,m_c_1,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,1.00,1.0,1.000,0.574785
425,5c4f5967aac8be0001716a66,m_c_2,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,1.00,1.0,1.000,0.829912
426,5c4f5967aac8be0001716a67,m_c_3,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,1.00,1.0,1.000,0.889956


In [70]:
config = pd.read_csv('/Users/shiyihe/Desktop/VIS_RESULT/zss2_writing_result_config.csv')
config_score=config[['participantId','final_check', 'strict_check', 'similarity','format']]
config_score['task_id'] = 'writing'
def add_score(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df['score'] = np.where(
        df['strict_check'] == 0,
        df['final_check'] * 0.25 + df['similarity'] * 0.5,
        df['final_check'] * 0.5  + df['similarity'] * 0.5
    )
    return df


new_df = add_score(config_score)
new_df['score_z'] = new_df.groupby('task_id')['score'] \
                  .transform(lambda x: (x - x.mean()) / x.std(ddof=0))
new_df=new_df[['participantId','task_id','score','score_z','format']]
new_df =new_df[new_df['task_id'] != 'overall']
new_df.to_csv('zss2_writing_config_score.csv',index=False)
new_df


/var/folders/hs/htjkkv116fv94g1llc3jj8jc0000gn/T/ipykernel_74777/1627917720.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  config_score['task_id'] = 'writing'


,participantId,task_id,score,score_z,format
0,670f9ddb0e32533c222c3031,writing,0.000000,-0.666112,json
1,671bdd06525adeea062a1d73,writing,0.000000,-0.666112,json
2,66912d7bf9d35c08f6919958,writing,0.000000,-0.666112,json
3,6818ca1523ece90f671e3601,writing,0.000000,-0.666112,json
4,6680d2279263349cc005956a,writing,0.880952,2.352823,json
...,...,...,...,...,...
102,67406c1303bd39d206ff2b50,writing,0.000000,-0.666112,xml
103,67f66486e35051403284eb5e,writing,0.000000,-0.666112,xml
104,60f7c71d1cffec381351b0b3,writing,0.281250,0.297703,xml
105,5fb4fcd1fce53c30649dd704,writing,0.000000,-0.666112,xml


In [54]:
new_df

,participantId,task_id,final_check,strict_check,similarity,format,score,score_z
0,670f9ddb0e32533c222c3031,1,0.00,0.00,0.000000,json,0.000000,-1.843495
1,671bdd06525adeea062a1d73,1,1.00,1.00,0.994382,json,0.997191,1.027876
2,66912d7bf9d35c08f6919958,1,0.00,0.00,0.000000,json,0.000000,-1.843495
3,6818ca1523ece90f671e3601,1,0.00,0.00,0.000000,json,0.000000,-1.843495
4,6680d2279263349cc005956a,1,1.00,1.00,0.983146,json,0.991573,1.011700
...,...,...,...,...,...,...,...,...
530,67f66486e35051403284eb5e,overall,1.00,0.00,0.820980,xml,0.660490,0.311170
531,680ebf035ab59d342f31726e,overall,1.00,1.00,0.928680,hjson,0.964340,1.437898
532,681374d36d7bd7ccb6f914f5,overall,1.00,1.00,0.000000,json5,0.500000,-0.283954
533,68138e7f96395f4c5f3a0a87,overall,0.50,0.25,0.479528,json,0.489764,-0.321911


In [48]:

df['score_z'] = df.groupby('task_id')['score'] \
                  .transform(lambda x: (x - x.mean()) / x.std(ddof=0))

# 2. 如果你只想看每个 participantId + task_id 对应的 z-score:
df


,participantId,task_id,score,score_z
0,A,1,0.8,6.798700e-16
1,B,1,0.6,-1.224745e+00
2,C,1,1.0,1.224745e+00
3,A,2,0.7,-6.798700e-16
4,B,2,0.5,-1.224745e+00
5,C,2,0.9,1.224745e+00


In [12]:
import pandas as pd

config = pd.read_csv('/Users/shiyihe/Desktop/VIS_RESULT/post_config.csv')
config_only = config[['participantId','task','difficulty','confidence','format']]
tabular = pd.read_csv('/Users/shiyihe/Desktop/VIS_RESULT/agg_tabular.csv')
tabular_only = tabular[['participantId','task','difficulty','confidence','format']]
# tabular = pd.read_csv('/Users/shiyihe/Desktop/VIS_RESULT/quiz_summary_tabular.csv')
# tabular_only = tabular[['participantId','task','num_wrong_attempts','correct','format']]

# quiz_data = pd.concat([config_only, tabular_only])
config_only.to_csv('dc_result_config.csv',index=False)
tabular_only.to_csv('dc_result_tabular.csv',index=False)

In [44]:
import pandas as pd

config = pd.read_csv('/Users/shiyihe/Desktop/VIS_RESULT/nasa_tidy_config.csv')
tabular = pd.read_csv('/Users/shiyihe/Desktop/VIS_RESULT/nasa_tidy_tabular.csv')
config_only = config[['participantId','task','score','format']]
tabular_only = tabular[['participantId','task','score','format']]
config_only.to_csv('5_nasa_tidy_config.csv',index=False)
tabular_only.to_csv('5_nasa_tidy_tabular.csv',index=False)
# # tabular_only = tabular[['participantId','task','num_wrong_attempts','correct','format']]

# # quiz_data = pd.concat([config_only, tabular_only])
# config_only.to_csv('dc_result_config.csv',index=False)
# tabular_only.to_csv('dc_result_tabular.csv',index=False)
# education = config[['participantId','q9']].rename(columns={'q9':'education'})

tabular


,participantId,format,startTime,endTime,duration_sec,task,score
0,64457bc906c125cebd4bf66b,json5,1746562427809,1746562451027,23.218,mental-demand,93
1,6697c8b6d9d7baa7edd8c4d3,hjson,1745933947314,1745933980430,33.116,mental-demand,90
2,67e25388d573ec2d5f1c8442,yaml,1746148170240,1746148206380,36.140,mental-demand,90
3,67e5cf90b79d0874e1cf16b0,toml,1746147492250,1746147510048,17.798,mental-demand,97
4,67d00f24466a67ba7e41228e,toml,1745933364792,1745933382986,18.194,mental-demand,79
...,...,...,...,...,...,...,...
637,67ab650c320eccf6141d4010,jsonc,1746147500734,1746147519483,18.749,frustration,62
638,65fad673a546ab14baf2f8d3,json5,1745932106163,1745932137658,31.495,frustration,75
639,67f1c7bd6f1c4a3d8deae02f,toml,1746146632834,1746146718970,86.136,frustration,10
640,67adb27bd5f5776fcb16da62,hjson,1746147510098,1746147559334,49.236,frustration,55


In [42]:
format_config = (
    tabular[['participantId','q12','format']]
     .rename(columns={'q12':'familiarity_format'})
)

# 确认一下列名
print(format_config.columns)
# >>> Index(['participantId','familiarity_format','format'], dtype='object')

# 2. 如果这一列还是字符串形式的字典，就 literal_eval
#    （如果它已经是 dict，就可以跳过这步）
format_config['familiarity_dict'] = format_config['familiarity_format'].apply(ast.literal_eval)

# 3. 把字典拆成多个列
fam_df = format_config['familiarity_dict'].apply(pd.Series)

#（可选）加个前缀
fam_df = fam_df.add_prefix('fam_')

# 4. 合并回去
result = pd.concat([
    format_config.drop(columns=['familiarity_format','familiarity_dict']),
    fam_df
], axis=1)
result.to_csv('format_tabular.csv',index=False)
result

Index(['participantId', 'familiarity_format', 'format'], dtype='object')


,participantId,format,fam_JSON,fam_JSONC,fam_JSON5,fam_HJSON,fam_YAML,fam_TOML,fam_XML
0,64457bc906c125cebd4bf66b,json5,Not familiar at all,Not familiar at all,Not familiar at all,Not familiar at all,Not familiar at all,Not familiar at all,Not familiar at all
1,6697c8b6d9d7baa7edd8c4d3,hjson,Heard of it but never used,Not familiar at all,Not familiar at all,Heard of it but never used,Not familiar at all,Not familiar at all,Used it a few times
2,67e25388d573ec2d5f1c8442,yaml,Not familiar at all,Used it a few times,Not familiar at all,Not familiar at all,Heard of it but never used,Not familiar at all,Used it a few times
3,67e5cf90b79d0874e1cf16b0,toml,Used it a few times,Heard of it but never used,Heard of it but never used,Not familiar at all,Not familiar at all,Not familiar at all,Heard of it but never used
4,67d00f24466a67ba7e41228e,toml,Heard of it but never used,Heard of it but never used,Heard of it but never used,Not familiar at all,Heard of it but never used,Heard of it but never used,Heard of it but never used
...,...,...,...,...,...,...,...,...,...
102,67ab650c320eccf6141d4010,jsonc,Expert,Used it a few times,Comfortable using it,Comfortable using it,Comfortable using it,Comfortable using it,Comfortable using it
103,65fad673a546ab14baf2f8d3,json5,Heard of it but never used,Heard of it but never used,Heard of it but never used,Heard of it but never used,Not familiar at all,Not familiar at all,Not familiar at all
104,67f1c7bd6f1c4a3d8deae02f,toml,Expert,Comfortable using it,Comfortable using it,Used it a few times,Comfortable using it,Comfortable using it,Comfortable using it
105,67adb27bd5f5776fcb16da62,hjson,Used it a few times,Heard of it but never used,Not familiar at all,Used it a few times,Not familiar at all,Used it a few times,Comfortable using it


In [29]:
education_config = config[['participantId','q9','format']].rename(columns={'q9':'education'})
education_config['education'] = (
    education_config['education']
      .str.replace("’", "'", regex=False)  # 把花引号换成直引号
      .str.strip()                         # 去掉前后空格
)

education_config['education'] = education_config['education'].replace({
    "Associate's":             "Other",
    "Some tertiary education": "Other",
    "Some college":            "Other"
})
education_config.to_csv('education_config.csv',index=False)
education_config['education'].value_counts()

education
Bachelor's     53
Master's       23
High School    17
PhD            11
Other           3
Name: count, dtype: int64

In [28]:
education_tabular = tabular[['participantId','q9','format']].rename(columns={'q9':'education'})
education_tabular['education'] = (
    education_tabular['education']
      .str.replace("’", "'", regex=False)  # 把花引号换成直引号
      .str.strip()                         # 去掉前后空格
)

education_tabular['education'] = education_tabular['education'].replace({
    "National Tech Diploma":             "Other",
    "College Diploma": "Other",
    "Student": "Other",
    "Certificate Program":            "Other"
})
education_tabular['education'].value_counts()
education_tabular.to_csv('education_tabular.csv',index=False)
education_tabular['education'].value_counts()

education
Bachelor's     62
Master's       30
High School    10
Other           4
PhD             1
Name: count, dtype: int64